## 경사 하강법을 이용한 얕은 신경망 학습

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [2]:
EPOCHS = 1000

## 네트워크 구조 정의
### 얕은 신경망
- 입력계층 :2  , 은닉 계층 : 128 ( 시그모이드 함수 ) , 출력계층 : 10 ( 소프트 맥스 함수 )

- init 에서 어떤 레이어를 사용할지 정의해줌
- call 은 모델이 실제로 call이 될때, 입력에서 출력으로 어떻게 되는지 정의하는 함수

In [21]:
class MyModel(tf.keras.Model): # keras의 model을 상속을 해서 클래스를 구현함. - 그리고 2가지 함수를 정의해야함
    def __init__(self):
        super(MyModel, self).__init__()  # 상속을 했으면, 상속한 함수를 initialize 해야함
        self.d1 = tf.keras.layers.Dense(128, input_dim = 2 , activation = 'sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation = 'softmax')
        # 입력데이터는 정의해줄 필요 x 왜냐하면 자기가 알아서 들어오기 때문 
        
        
    def call(self, x, training = None , mask = None):
        x = self.d1(x) # 은닉계층이 됨
        return self.d2(x) #출력계층이됨

## 학습루프 정의

- tf의 오토그래프를 이용해서 최적화를 해줄것이다. 
- @tf.function : 이걸 사용하면 trian_step안에 있는 tensor 연산들이 파이썬 문법들로 계산을 하는데, tf.function 을 통해 tensor의 최적화를 해줌

- train_step 에서 가장 중요한 것은, gradient 를 계산해서, 그라디언트를 옵티마이저에 넣어서 학습을 진행하는 것.

In [22]:
@tf.function 
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape: # 여기 안에서 계산된것은 gradient가 계산되고 tape 안에 들어가 있음
        predictions = model(inputs)  # 모델안에 인풋을 넣어서 예측값을 계산함 
        loss= loss_object(labels, predictions ) # loss 를 계산하는데, loss_object에다가 정답이랑 예측값을 넣어서 계산
        # 그러면, gradient가 계산된 것들이 tape안으로 들어가게 됨. 
    
    gradients = tape.gradient(loss, model.trainable_variables ) # tape에서 gradient를 계산을 하는데,  
    # 첫번째 변수를 두번째 변수로 미분해서 gradient를 계산하는 것이 gradients 에 들어가게 된다.
    # loss는 스칼라이며, model에 있는 모든 trainable variables 에 미분을 해준다. 
    # df(x)/dx 를 하는 모양이며, 스칼라를 벡터로 미분하는 것이므로, gradients는 벡터가 된다. ( 편미분한게 들어감 )
    # 이 벡터를 가지고 optimizer를 이용해서 gradient를 적용해줌
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # 학습될것을 zip해서 넣어줌. 각 gradients에 대해 각 trainable_variables 들이 gradient가 적용되서 optimizer로 학습이 됨
    
    train_loss(loss) # train_loss가 loss를 종합해주는 역할을 함
    train_metric(labels, predictions ) #trian_metric을 이용해서 정답과 prediction을 비교해서 평가 지표로 계산을헤줌
    # 여기서는 accuray를 사용할 것 . 몇퍼센트가 정확한지 확인 
    

## 데이터 셋 생성, 전처리
- 따로 데이터가 주어진게 아니라 np로 만들어 보기! 
- 위의 tensorflow로 모델을 만들때, 설정을 해주지 않는다면 자동적으로 float32가됨. 
- 데이터 셋도 float64가아닌 32로 만들어 줘야 . 오류가 나지 않는다! 

In [23]:
np.random.seed(0) # random seed를 고정하면,  항상 실행해도 동일한 값이 나옴

pts = list()  # 입력값
labels = list()  # 출력값
# 데이터 셋을 구성하는데 있어서, 10개의 클래스를 가지는 포인터를 생성할 것이다.

center_pts = np.random.uniform(-8.0, 8.0 ,(10,2))  # 이렇게 되면 값이 np.float64가 됨. 

for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))  # 중앙 포인트를 기준으로 랜덤하게 동그랗게 퍼지게 됨
        labels.append(label)  # 총 10개의 class를 생성해줌
        
pts = np.stack(pts, axis= 0).astype(np.float32) # 리스트형태인 pts를 stack을 이용해서 넘파이 형태로 만들어줌 
labels = np.stack(labels, axis=0)

train_dataset = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)
# 입력과 정답을 묶어서 넣어주면, train dataset으로 합쳐줨 
# 데이터 1000개까지 섞음 
# cpu가 학습이 안되면  batch사이즈를 줄이면 됨! 


## 모델 생성

In [24]:
model = MyModel()

## 손실함수와 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [25]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() 
optimizer = tf.keras.optimizers.Adam()

## 평가지표설정
### Accuracy

In [26]:
train_loss = tf.keras.metrics.Mean(name = 'trian_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuaracy')

### 학습루프

In [29]:
for epoch in range(EPOCHS):
    for x , label in train_dataset: # 배치를 32이로 해놨기 때문에 32개의 인풋과 정답이 계속 나오게 됨
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch {} , Loss: {}, Accuracy : {}'
    print(template.format(epoch +1 , train_loss.result(), train_accuracy.result()*100))

Epoch 1 , Loss: 0.5888902544975281, Accuracy : 84.38142395019531
Epoch 2 , Loss: 0.5850707292556763, Accuracy : 84.43940734863281
Epoch 3 , Loss: 0.5813665390014648, Accuracy : 84.49577331542969
Epoch 4 , Loss: 0.5777169466018677, Accuracy : 84.5547103881836
Epoch 5 , Loss: 0.5741634964942932, Accuracy : 84.61071014404297
Epoch 6 , Loss: 0.5707200169563293, Accuracy : 84.66120910644531
Epoch 7 , Loss: 0.5673087239265442, Accuracy : 84.71170043945312
Epoch 8 , Loss: 0.5640558004379272, Accuracy : 84.76087951660156
Epoch 9 , Loss: 0.5608530640602112, Accuracy : 84.81520080566406
Epoch 10 , Loss: 0.5576314330101013, Accuracy : 84.8580322265625
Epoch 11 , Loss: 0.5545216202735901, Accuracy : 84.9035415649414
Epoch 12 , Loss: 0.5514828562736511, Accuracy : 84.96026611328125
Epoch 13 , Loss: 0.5485026240348816, Accuracy : 85.00463104248047
Epoch 14 , Loss: 0.5455869436264038, Accuracy : 85.0467300415039
Epoch 15 , Loss: 0.542660653591156, Accuracy : 85.09496307373047
Epoch 16 , Loss: 0.53985

Epoch 124 , Loss: 0.3940878212451935, Accuracy : 87.37670135498047
Epoch 125 , Loss: 0.3934171795845032, Accuracy : 87.38764190673828
Epoch 126 , Loss: 0.392770916223526, Accuracy : 87.39437103271484
Epoch 127 , Loss: 0.3921521008014679, Accuracy : 87.40409851074219
Epoch 128 , Loss: 0.3915221691131592, Accuracy : 87.4137191772461
Epoch 129 , Loss: 0.3908863961696625, Accuracy : 87.42324829101562
Epoch 130 , Loss: 0.39022520184516907, Accuracy : 87.4341812133789
Epoch 131 , Loss: 0.3895835876464844, Accuracy : 87.44500732421875
Epoch 132 , Loss: 0.3889572322368622, Accuracy : 87.45622253417969
Epoch 133 , Loss: 0.3883283734321594, Accuracy : 87.46782684326172
Epoch 134 , Loss: 0.38773074746131897, Accuracy : 87.47685241699219
Epoch 135 , Loss: 0.38715696334838867, Accuracy : 87.48284912109375
Epoch 136 , Loss: 0.3865545988082886, Accuracy : 87.48976135253906
Epoch 137 , Loss: 0.3859756886959076, Accuracy : 87.49805450439453
Epoch 138 , Loss: 0.3854106664657593, Accuracy : 87.5077285766

Epoch 247 , Loss: 0.3433685302734375, Accuracy : 88.0980453491211
Epoch 248 , Loss: 0.3430849015712738, Accuracy : 88.10246276855469
Epoch 249 , Loss: 0.34281685948371887, Accuracy : 88.1065444946289
Epoch 250 , Loss: 0.3425407111644745, Accuracy : 88.110595703125
Epoch 251 , Loss: 0.3422844111919403, Accuracy : 88.11431884765625
Epoch 252 , Loss: 0.34202006459236145, Accuracy : 88.11894226074219
Epoch 253 , Loss: 0.34174808859825134, Accuracy : 88.12198638916016
Epoch 254 , Loss: 0.34151774644851685, Accuracy : 88.12718200683594
Epoch 255 , Loss: 0.34126660227775574, Accuracy : 88.12771606445312
Epoch 256 , Loss: 0.34101420640945435, Accuracy : 88.13255310058594
Epoch 257 , Loss: 0.34077003598213196, Accuracy : 88.13459777832031
Epoch 258 , Loss: 0.3405168354511261, Accuracy : 88.13908386230469
Epoch 259 , Loss: 0.34027236700057983, Accuracy : 88.14200592041016
Epoch 260 , Loss: 0.3400440812110901, Accuracy : 88.1461410522461
Epoch 261 , Loss: 0.33979541063308716, Accuracy : 88.148422

Epoch 372 , Loss: 0.319472074508667, Accuracy : 88.45435333251953
Epoch 373 , Loss: 0.31933051347732544, Accuracy : 88.45626831054688
Epoch 374 , Loss: 0.31919869780540466, Accuracy : 88.4586181640625
Epoch 375 , Loss: 0.3190527856349945, Accuracy : 88.46141815185547
Epoch 376 , Loss: 0.3189256191253662, Accuracy : 88.4621810913086
Epoch 377 , Loss: 0.318784236907959, Accuracy : 88.46472930908203
Epoch 378 , Loss: 0.31864309310913086, Accuracy : 88.46614837646484
Epoch 379 , Loss: 0.3184969127178192, Accuracy : 88.46868133544922
Epoch 380 , Loss: 0.3183755874633789, Accuracy : 88.47052764892578
Epoch 381 , Loss: 0.31822243332862854, Accuracy : 88.47148132324219
Epoch 382 , Loss: 0.3180880546569824, Accuracy : 88.47265625
Epoch 383 , Loss: 0.3179560899734497, Accuracy : 88.47537994384766
Epoch 384 , Loss: 0.31781551241874695, Accuracy : 88.47653198242188
Epoch 385 , Loss: 0.3176644444465637, Accuracy : 88.47834777832031
Epoch 386 , Loss: 0.3175652027130127, Accuracy : 88.4803695678711
E

Epoch 495 , Loss: 0.3053930103778839, Accuracy : 88.6761703491211
Epoch 496 , Loss: 0.3052998483181, Accuracy : 88.6792221069336
Epoch 497 , Loss: 0.30521124601364136, Accuracy : 88.6803207397461
Epoch 498 , Loss: 0.3051227033138275, Accuracy : 88.68229675292969
Epoch 499 , Loss: 0.3050253093242645, Accuracy : 88.6832046508789
Epoch 500 , Loss: 0.30494293570518494, Accuracy : 88.68517303466797
Epoch 501 , Loss: 0.3048456907272339, Accuracy : 88.6864242553711
Epoch 502 , Loss: 0.3047555983066559, Accuracy : 88.68767547607422
Epoch 503 , Loss: 0.30466437339782715, Accuracy : 88.6887435913086
Epoch 504 , Loss: 0.30457115173339844, Accuracy : 88.69068908691406
Epoch 505 , Loss: 0.3044804036617279, Accuracy : 88.6917495727539
Epoch 506 , Loss: 0.3043902516365051, Accuracy : 88.69367218017578
Epoch 507 , Loss: 0.3043070137500763, Accuracy : 88.69646453857422
Epoch 508 , Loss: 0.30422183871269226, Accuracy : 88.6978530883789
Epoch 509 , Loss: 0.3041238784790039, Accuracy : 88.6995849609375
Ep

Epoch 619 , Loss: 0.29591548442840576, Accuracy : 88.84921264648438
Epoch 620 , Loss: 0.2958480715751648, Accuracy : 88.8505859375
Epoch 621 , Loss: 0.2957734167575836, Accuracy : 88.85196685791016
Epoch 622 , Loss: 0.29569992423057556, Accuracy : 88.85363006591797
Epoch 623 , Loss: 0.29562580585479736, Accuracy : 88.85514068603516
Epoch 624 , Loss: 0.29555392265319824, Accuracy : 88.85607147216797
Epoch 625 , Loss: 0.29548391699790955, Accuracy : 88.85699462890625
Epoch 626 , Loss: 0.2954216003417969, Accuracy : 88.85835266113281
Epoch 627 , Loss: 0.29536691308021545, Accuracy : 88.85956573486328
Epoch 628 , Loss: 0.2953099310398102, Accuracy : 88.86033630371094
Epoch 629 , Loss: 0.29524683952331543, Accuracy : 88.86111450195312
Epoch 630 , Loss: 0.2952086925506592, Accuracy : 88.86231231689453
Epoch 631 , Loss: 0.2951447367668152, Accuracy : 88.8637924194336
Epoch 632 , Loss: 0.29508140683174133, Accuracy : 88.864990234375
Epoch 633 , Loss: 0.2950265407562256, Accuracy : 88.866600036

Epoch 742 , Loss: 0.28889262676239014, Accuracy : 89.00537109375
Epoch 743 , Loss: 0.28883951902389526, Accuracy : 89.006591796875
Epoch 744 , Loss: 0.288784384727478, Accuracy : 89.0081787109375
Epoch 745 , Loss: 0.28872838616371155, Accuracy : 89.00965118408203
Epoch 746 , Loss: 0.2886708378791809, Accuracy : 89.0113525390625
Epoch 747 , Loss: 0.2886137068271637, Accuracy : 89.01243591308594
Epoch 748 , Loss: 0.2885650396347046, Accuracy : 89.0138931274414
Epoch 749 , Loss: 0.28852134943008423, Accuracy : 89.01510620117188
Epoch 750 , Loss: 0.28847071528434753, Accuracy : 89.01667022705078
Epoch 751 , Loss: 0.2884232997894287, Accuracy : 89.01811218261719
Epoch 752 , Loss: 0.2883686423301697, Accuracy : 89.0194320678711
Epoch 753 , Loss: 0.2883132994174957, Accuracy : 89.02098846435547
Epoch 754 , Loss: 0.28826358914375305, Accuracy : 89.02193450927734
Epoch 755 , Loss: 0.28822094202041626, Accuracy : 89.02263641357422
Epoch 756 , Loss: 0.28817594051361084, Accuracy : 89.023818969726

Epoch 866 , Loss: 0.2833316922187805, Accuracy : 89.15352630615234
Epoch 867 , Loss: 0.2832900583744049, Accuracy : 89.15485382080078
Epoch 868 , Loss: 0.2832448184490204, Accuracy : 89.15586853027344
Epoch 869 , Loss: 0.283199667930603, Accuracy : 89.15740966796875
Epoch 870 , Loss: 0.28317171335220337, Accuracy : 89.15819549560547
Epoch 871 , Loss: 0.28313055634498596, Accuracy : 89.1593017578125
Epoch 872 , Loss: 0.28308603167533875, Accuracy : 89.16030883789062
Epoch 873 , Loss: 0.2830415964126587, Accuracy : 89.16130065917969
Epoch 874 , Loss: 0.28301265835762024, Accuracy : 89.16218566894531
Epoch 875 , Loss: 0.2829764783382416, Accuracy : 89.16329193115234
Epoch 876 , Loss: 0.2829355299472809, Accuracy : 89.1639633178711
Epoch 877 , Loss: 0.2828981876373291, Accuracy : 89.16548156738281
Epoch 878 , Loss: 0.28285786509513855, Accuracy : 89.16646575927734
Epoch 879 , Loss: 0.2828145921230316, Accuracy : 89.16798400878906
Epoch 880 , Loss: 0.28278666734695435, Accuracy : 89.1692810

Epoch 987 , Loss: 0.2790182828903198, Accuracy : 89.28252410888672
Epoch 988 , Loss: 0.27898332476615906, Accuracy : 89.2839584350586
Epoch 989 , Loss: 0.27894771099090576, Accuracy : 89.2850112915039
Epoch 990 , Loss: 0.27892008423805237, Accuracy : 89.2863540649414
Epoch 991 , Loss: 0.2788854241371155, Accuracy : 89.28739929199219
Epoch 992 , Loss: 0.27884942293167114, Accuracy : 89.28882598876953
Epoch 993 , Loss: 0.27881500124931335, Accuracy : 89.28968048095703
Epoch 994 , Loss: 0.2787836492061615, Accuracy : 89.29082489013672
Epoch 995 , Loss: 0.2787488102912903, Accuracy : 89.29186248779297
Epoch 996 , Loss: 0.27871549129486084, Accuracy : 89.29337310791016
Epoch 997 , Loss: 0.27870064973831177, Accuracy : 89.29403686523438
Epoch 998 , Loss: 0.27866506576538086, Accuracy : 89.29469299316406
Epoch 999 , Loss: 0.27862757444381714, Accuracy : 89.2952651977539
Epoch 1000 , Loss: 0.2785913646221161, Accuracy : 89.2960205078125


## 데이터셋 및 학습 파라미터 저장
- np.savez_compressed  : 여러개의 넘파이오브젝트를 한번에 저장할 수 있으며, 압축해서 저장할 수 있다. 

In [30]:
np.savez_compressed('np_dataset.npz', inputs = pts, labels = labels)

W_h, b_h = model.d1.get_weights() 
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('parameters.npz', W_h = W_h, b_h =b_h, W_o =W_o,b_o=b_o )